In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df_tender_2018 = pd.read_csv("../data/releases_2018/tender_2018.csv")
df_tender_2019 = pd.read_csv("../data/releases_2019/tender_2019.csv")
df_tender_2020 = pd.read_csv("../data/releases_2020/tender_2020.csv")
df_tender_2021 = pd.read_csv("../data/releases_2021/tender_2021.csv")
df_tender = pd.concat([df_tender_2018, df_tender_2019, df_tender_2020, df_tender_2021], axis=0)

c:\Users\jhony\miniconda3\envs\icd\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (12,13,14,15,16,18,19,20,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df_award_2018 = pd.read_csv("../data/releases_2018/awards_2018.csv")
df_award_2019 = pd.read_csv("../data/releases_2019/awards_2019.csv")
df_award_2020 = pd.read_csv("../data/releases_2020/awards_2020.csv")
df_award_2021 = pd.read_csv("../data/releases_2021/awards_2021.csv")
df_award = pd.concat([df_award_2018, df_award_2019, df_award_2020, df_award_2021], axis=0)

c:\Users\jhony\miniconda3\envs\icd\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (4,5,6,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\jhony\miniconda3\envs\icd\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (4,6,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_contract_2018 = pd.read_csv("../data/releases_2018/contracts_2018.csv")
df_contract_2019 = pd.read_csv("../data/releases_2019/contracts_2019.csv")
df_contract_2020 = pd.read_csv("../data/releases_2020/contracts_2020.csv")
df_contract_2021 = pd.read_csv("../data/releases_2021/contracts_2021.csv")
df_contract = pd.concat([df_contract_2018, df_contract_2019, df_contract_2020, df_contract_2021], axis=0)

c:\Users\jhony\miniconda3\envs\icd\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (7,8,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_supplier_2018 = pd.read_csv("../data/releases_2018/suppliers_2018.csv")
df_supplier_2019 = pd.read_csv("../data/releases_2019/suppliers_2019.csv")
df_supplier_2020 = pd.read_csv("../data/releases_2020/suppliers_2020.csv")
df_supplier_2021 = pd.read_csv("../data/releases_2021/suppliers_2021.csv")
df_supplier = pd.concat([df_supplier_2018, df_supplier_2019, df_supplier_2020, df_supplier_2021], axis=0)

In [8]:
feature1_tender = df_tender[["ocid", "procuringEntity_name", "value_amount", "procurementMethod"]]
feature1_award = df_award[["ocid", "id", "amount", "status"]]
feature1_contract = df_contract[["ocid", "id", "amount"]]
feature1_supplier = df_supplier[["award_id", "name"]]

In [9]:
main_df = feature1_award.merge(feature1_contract, left_on="ocid", right_on="ocid", how="left", suffixes=("_aw", "_ctr"))
main_df = main_df.merge(feature1_tender, left_on="ocid", right_on="ocid", how="left")
main_df = main_df.merge(feature1_supplier, left_on="id_aw", right_on="award_id", how="left")

In [10]:
main_df.columns

Index(['ocid', 'id_aw', 'amount_aw', 'status', 'id_ctr', 'amount_ctr',
       'procuringEntity_name', 'value_amount', 'procurementMethod', 'award_id',
       'name'],
      dtype='object')

In [11]:
main_df.shape

(1547451, 11)

In [12]:
filtro1 = ~(main_df["amount_ctr"].isnull()) & ~(main_df["amount_aw"].isnull())
main_df = main_df[filtro1]

In [13]:
main_df.shape

(706303, 11)

In [14]:
main_df["diferencia_absoluta"] = np.abs(main_df["amount_aw"] - main_df["amount_ctr"])
main_df["diferencia_relativa"] = np.abs(main_df["diferencia_absoluta"] / main_df["amount_aw"])
main_df.head()

,ocid,id_aw,amount_aw,status,id_ctr,amount_ctr,procuringEntity_name,value_amount,procurementMethod,award_id,name,diferencia_absoluta,diferencia_relativa
1,ocds-5wno2w-SIE-008-HMHP-2018-11080,1236409-SIE-008-HMHP-2018,27699.00,NaN,526405-SIE-008-HMHP-2018,27699.00,HOSPITAL MATILDE HIDALGO DE PROCEL,NaN,open,1236409-SIE-008-HMHP-2018,SALAZAR SALAZAR LUIS ALBERTO,0.00,0.00
2,ocds-5wno2w-SIE-CONAFIPS-03-2018-448191,1374768-SIE-CONAFIPS-03-2018,10300.93,NaN,578390-SIE-CONAFIPS-03-2018,10300.93,CORPORACIÓN NACIONAL DE FINANZAS POPULARES Y S...,NaN,open,1374768-SIE-CONAFIPS-03-2018,ITSEGUINFO CIA. LTDA.,0.00,0.00
3,ocds-5wno2w-SIE-EPP-2016233-018-253178,1375948-SIE-EPP-2016233-018,192000.00,NaN,589838-SIE-EPP-2016233-018,192000.00,Empresa Pública de hidrocarburos del Ecuador E...,NaN,open,1375948-SIE-EPP-2016233-018,IASA S.A.,0.00,0.00
4,ocds-5wno2w-SIE-EPP-674Y678-1-18-253178,1390713-SIE-EPP-674Y678-1-18,65800.00,NaN,611465-SIE-EPP-674Y678-1-18,65800.00,Empresa Pública de hidrocarburos del Ecuador E...,NaN,open,1390713-SIE-EPP-674Y678-1-18,VELASTEGUI LASSO BOLIVAR ALEJANDRO,0.00,0.00
14,ocds-5wno2w-RE-HEJCA-F-66-2018-87497,3416132-RE-HEJCA-F-66-2018,14810.82,NaN,565301-RE-HEJCA-F-66-2018,14810.82,HOSPITAL DE ESPECIALIDADES JOSÉ CARRASCO ARTEAGA,14810.82,direct,3416132-RE-HEJCA-F-66-2018,QUIFATEX S.A.,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547354,ocds-5wno2w-REOALC-GADMANTA01-21-2668,5151471-REOALC-GADMANTA01-21,268000.00,NaN,2115646-REOALC-GADMANTA01-21,268000.00,GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DE...,268000.00,direct,5151471-REOALC-GADMANTA01-21,YEROVI LOPEZ ROSSYENI MONSERRAT,0.00,0.00
1547447,ocds-5wno2w-MCBS-DNA-005-2021-178618,4862272-MCBS-DNA-005-2021,28823.77,NaN,2115899-MCBS-DNA-005-2021,28823.77,DIRECCION NACIONAL DE ANTINARCOTICOS,28823.77,selective,4862272-MCBS-DNA-005-2021,NARVAEZ BAHAMONDE JORGE ENRIQUE,0.00,0.00
1547448,ocds-5wno2w-MCS-CCFFAA-002-ABR21-2539,4744578-MCS-CCFFAA-002-ABR21,7589.29,NaN,2072552-MCS-CCFFAA-002-ABR21,7589.29,COMANDO CONJUNTO,7589.29,selective,4744578-MCS-CCFFAA-002-ABR21,SUMAKTOUR TRAVEL & TOURS CIA. LTDA.,0.00,0.00
1547449,ocds-5wno2w-RE-GADMCN-17-21-LRG-67188,5302403-RE-GADMCN-17-21-LRG,30590.89,NaN,2115789-RE-GADMCN-17-21-LRG,30590.89,GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DE...,30590.89,direct,5302403-RE-GADMCN-17-21-LRG,IASA-SERVICIO S.A.,0.00,0.00


In [15]:
#Empresa, diferenciaRelativaPromedio, numeroProcesos, porcentajeAdjudicacion, cantidadEmpresaDiferentes

In [16]:
percentaje = main_df.groupby(["procuringEntity_name", "name"])["id_aw"].count().reset_index()

In [17]:
main_df = main_df.groupby("procuringEntity_name").agg({
    "diferencia_relativa" : "mean",
    "diferencia_absoluta" : "mean",
}).reset_index()

In [18]:

main_df["total_procesos"] = percentaje.groupby(["procuringEntity_name"])["id_aw"].sum().reset_index()["id_aw"]
main_df["unicos"] = percentaje.groupby(["procuringEntity_name"])["id_aw"].count().reset_index()["id_aw"]
main_df["porcentaje_unicos"] = main_df["unicos"] / main_df["total_procesos"]

In [19]:
main_df.head()

,procuringEntity_name,diferencia_relativa,diferencia_absoluta,total_procesos,unicos,porcentaje_unicos
0,\t GOBIERNO AUTONOMO PARROQUIAL DE SINAI,0.00,0.00,1,1,1.00
1,GOBIERNO AUTONOMO DESCENTRALIZADO DE LA PARRO...,0.00,0.00,6,6,1.00
2,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,0.00,0.00,1,1,1.00
3,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,0.00,0.00,3,3,1.00
4,ACCION SOCIAL DEL GOBIERNO AUTONOMO DESCENTRAL...,0.00,0.00,16,15,0.94
...,...,...,...,...,...,...
2842,junta parroquial la tola,0.00,0.00,1,1,1.00
2843,juntaparroquialsalango,0.00,0.00,1,1,1.00
2844,miduvi Santo Domingo de los TSachilas,0.00,0.00,1,1,1.00
2845,museo y parque arqueologico pumapungo,5.83,1093.17,206,10,0.05


In [20]:
main_df.columns = ["Licitante", "Diferencia relativa Promedio", "Diferencia absoluta promedio", "Total Procesos", "Empresas Unicas", "Porcentaje Empresas Unicas"]
main_df.head()

,Licitante,Diferencia relativa Promedio,Diferencia absoluta promedio,Total Procesos,Empresas Unicas,Porcentaje Empresas Unicas
0,\t GOBIERNO AUTONOMO PARROQUIAL DE SINAI,0.00,0.00,1,1,1.00
1,GOBIERNO AUTONOMO DESCENTRALIZADO DE LA PARRO...,0.00,0.00,6,6,1.00
2,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,0.00,0.00,1,1,1.00
3,GOBIERNO AUTONOMO DESCENTRALIZADO PARROQUIAL ...,0.00,0.00,3,3,1.00
4,ACCION SOCIAL DEL GOBIERNO AUTONOMO DESCENTRAL...,0.00,0.00,16,15,0.94
